In [48]:
import sys
from scipy import misc

if sys.version_info >= (3, 2):
    from functools import lru_cache
else:
    from repoze.lru import lru_cache
    
if hasattr(scipy.misc, 'comb'):
    scipy_comb = scipy.misc.comb
else:
    import scipy.special
    scipy_comb = scipy.special.comb
   

@lru_cache(maxsize=1024)
def comb(n, k, exact=False):
    return scipy_comb(n, k, exact=exact)


def rank(s, n):
    """
    Returns index of the combination s in (N,K)
    https://computationalcombinatorics.wordpress.com/2012/09/10/ranking-and-unranking-of-combinations-and-permutations/
    :param s:
    :return:
    """
    k = len(s)
    r = 0
    for i in range(0, k):
        for v in range(s[i-1]+1 if i > 0 else 0, s[i]):
            r += comb_cached(n - v - 1, k - i - 1)
    return r


@lru_cache(maxsize=8192)
def unrank(i, n, k):
    """
    returns the i-th combination of k numbers chosen from 0,2,...,n-1, indexing from 0
    """
    c = []
    r = i+0
    j = 0
    for s in range(1, k+1):
        cs = j+1

        while True:
            if n-cs < 0:
                raise ValueError('Invalid index')
            decr = comb(n-cs, k-s)
            if r > 0 and decr == 0:
                raise ValueError('Invalid index')
            if r - decr >= 0:
                r -= decr
                cs += 1
            else:
                break

        c.append(cs-1)
        j = cs
    return c


def partition_space(hw, bits, partitions):
    comb_space = int(comb(bits, hw))
    total_size = bits * comb_space
    total_size_mb = total_size / 8 / 1024 / 1024
    
    chunk_size = int(comb_space // partitions)
    print('HW: %s, Block size: %s bits\n  Total combinations: %s = %.2f MB stream size' 
          % (hw, bits, comb_space, total_size_mb))
    print('\nPartitioning to %s parts, partition size: %s, sequence size: %.2f MB\n' 
          % (partitions, chunk_size, bits * chunk_size / 8 / 1024 / 1024))
    
    res = []
    for i in range(partitions):
        offset = i*chunk_size
        state = unrank(offset, bits, hw)
        res.append(state)
        print('  state[%s]: %s' % (i, state))
    return chunk_size, res

partition_space(4, 128, 4)

HW: 4, Block size: 128 bits
  Total combinations: 10668000 = 162.78 MB stream size

Partitioning to 4 parts, partition size: 2667000, sequence size: 40.70 MB

  state[0]: [0, 1, 2, 3]
  state[1]: [8, 55, 92, 114]
  state[2]: [20, 25, 69, 91]
  state[3]: [37, 39, 63, 70]


/home/xklinec/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `comb` is deprecated!
Importing `comb` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.comb` instead.


(2667000, [[0, 1, 2, 3], [8, 55, 92, 114], [20, 25, 69, 91], [37, 39, 63, 70]])

In [42]:
unrank(10668000-1, 128, 4)

/home/xklinec/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `comb` is deprecated!
Importing `comb` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.comb` instead.


[124, 125, 126, 127]

In [1]:
from rtt_tools import dump_data
L = dump_data.Loader()

In [2]:
L.break_exp('SECMARGINPAPER16_hw_seed_1fe40505e131963c_10MiB_key_1212313_off_1212_der_121233__AES_r03_b16.bin')

Einfo(id=16, m='hw', s='1fe40505e131963c', si=10485760, osi='10MiB', fname=None, fr=None, fb=None, k='1212313', off='1212', der='121233')
('16', 'hw', '1fe40505e131963c', '10MiB', '1212313', '1212', '121233', 'AES_r03_b16', '.bin')
AES_r03_b16


Einfo(id=16, m='hw', s='1fe40505e131963c', si=10485760, osi='10MiB', fname='AES', fr=3, fb='16', k='1212313', off='1212', der='121233')

In [5]:
L.break_exp('SECMARGINPAPER16_hw_seed_1fe40505e131963c_10MiB__AES_r03_b16.bin')

Einfo(id=16, m='hw', s='1fe40505e131963c', si=10485760, osi='10MiB', fname=None, fr=None, fb=None, k=None, off=None, der=None)
('16', 'hw', '1fe40505e131963c', '10MiB', None, None, None, 'AES_r03_b16', '.bin')
AES_r03_b16


Einfo(id=16, m='hw', s='1fe40505e131963c', si=10485760, osi='10MiB', fname='AES', fr=3, fb='16', k=None, off=None, der=None)